env test

In [ ]:
# 파이썬 설치 확인
# > python --version

# 파이썬 가상 환경 생성
# > python -m venv myenv

# 파이썬 가상 환경 활성화
# (Windows) > myenv\Scripts\activate
# (macOs / Linux) > source myenv/bin/activate

# 파이썬 패키지 설치
# > pip install requests
# > pip install flask
# > pip install mysql-connector-python

# Flask 앱 생성
from flask import Flask

app = Flask(__name__)

@app.route('/')
def flask_test():
    return 'flask test'

if __name__ == '__main__':
    app.run()
# > python test_env.py

# 파이썬 가상 환경 종료
# > deactivate


# > git init
# > git status
# > git remote -v
# > git branch 브랜치이름
# > git checkout 브랜치이름
# > git branch
# > git add 파일이름
# > git add .
# > git commit -m "커밋 메시지"
# > git push origin 브랜치이름

flask api test

In [ ]:
# > myenv\Scripts\activate
# > python test_api.py
# http://127.0.0.1:5000/test1?param1=12&param2=34 접속
# http://127.0.0.1:5000/test2?param1=56&param2=78 접속
from flask import Flask, request, Response, jsonify

app = Flask(__name__)

@app.route('/test1', methods=['GET'])
def get_params1():
    param1 = request.args.get('param1')
    param2 = request.args.get('param2')
    if param1 and param2:
        response = {
            'function': 'get_params1',
            'return1': param1,
            'return2': param2,
            'message': 'ok'
        }
        return jsonify(response), 200
    return Response('Unauthorized', 400)

@app.route('/test2', methods=['GET'])
def get_params2():
    param1 = request.args.get('param1')
    param2 = request.args.get('param2')
    if param1 and param2:
        response = {
            'function': 'get_params2',
            'return1': param1,
            'return2': param2,
            'message': 'ok'
        }
        return jsonify(response), 200
    return Response('Unauthorized', 400)

if __name__ == '__main__':
    app.run()


mysql test

In [ ]:
# > pip install mysql-connector-python
import mysql.connector

db_connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="root",
    database="safety_route"
)

cursor = db_connection.cursor()

In [ ]:
select_query = "SELECT * FROM CCTV"
cursor.execute(select_query)
result = cursor.fetchall()

for row in result[:10]:
    print(row)
for row in result [-10:]:
    print(row)

In [ ]:
cursor.close()
db_connection.close()

OSRM request test

In [ ]:
# > pip insatll requests
import requests

def get_route(depart_x, depart_y, arrive_x, arrive_y):
    depart_x, depart_y, arrive_x, arrive_y = str(depart_x), str(depart_y), str(arrive_x), str(arrive_y)
    request_url = "http://localhost:5000/route/v1/foot/"+depart_x+","+depart_y+";"+arrive_x+","+arrive_y+"?alternatives=3&steps=true"
    return requests.get(url=request_url)


In [ ]:
# OSRM docker 실행 필요

route = get_route(126.63486162967031, 37.376924147295455, 126.64251548843, 37.38308798021501)
print(route)
print(route.json())
print(route.json()['code'])

In [ ]:
from pprint import pprint

pprint(route.json())

In [ ]:
import requests
import traceback
from pprint import pprint

try:
    depart_x, depart_y, arrive_x, arrive_y = 126.63486, 37.37692, 126.64251, 37.38308
    depart_x, depart_y, arrive_x, arrive_y = str(depart_x), str(depart_y), str(arrive_x), str(arrive_y)
    OSRM_url = "http://localhost:5000/route/v1/foot/"+depart_x+","+depart_y+";"+arrive_x+","+arrive_y+"?alternatives=3&steps=true"
    OSRM_response = requests.get(url=OSRM_url)
    if OSRM_response.status_code == 404:
        print('404')
    else:
        routes = OSRM_response.json()['routes']
        num_route = len(routes)
        route_info = [{} for _ in range(num_route)]
        for i in range(num_route):
            route_info[i]['distance'] = routes[i]['distance']
            route_info[i]['geometry'] = routes[i]['geometry']
            route_info[i]['nodes'] = [[depart_x, depart_y]]
            route_info[i]['near_cctv'] = []
            steps = routes[i]['legs'][0]['steps']
            for step in steps:
                route_info[i]['nodes'].append(step['maneuver']['location'])
            route_info[i]['nodes'].append([arrive_x, arrive_y])
        pprint(route_info)
except Exception as e:
    err = traceback.format_exc()
    print(err)